# **Topic Modeling on Full dataset sample**

In [12]:
import pandas as pd
import re

import time
import math
import re
# import textblob
# from textblob import TextBlob
import pandas as pd
import numpy as np

import nltk as nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import string
import gensim
from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import CoherenceModel

# import pyLDAvis.gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from gensim.utils import simple_preprocess
pyLDAvis.enable_notebook()

import itertools

In [13]:
# Imports the Google Cloud client library
from google.cloud import storage
# Instantiates a client
storage_client = storage.Client()

# The name for the new bucket
bucket_name = "nlp_final_project_kshitijm"

# Creates the new bucket
bucket = storage_client.bucket(bucket_name)
print(f"Bucket {bucket.name} connected.")

Bucket nlp_final_project_kshitijm connected.


In [14]:
%pip install pandarallel
import multiprocessing

num_processors = multiprocessing.cpu_count()
print(f'Available CPUs: {num_processors}')

import pandarallel
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=num_processors-1, use_memory_fs=False)

num_processors = multiprocessing.cpu_count()
workers = num_processors-1

Note: you may need to restart the kernel to use updated packages.
Available CPUs: 16
INFO: Pandarallel will run on 15 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


### Loading Sample Data

In [15]:
df_news_samp=pd.read_parquet('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data2_tokens_samp.parquet')
df_news_samp.head()

,Unnamed: 0,Unnamed: 0.1,url,date,language,title,text,cleaned_text,article_source,clean_title,title_tokens,cleaned_text_tokens
189525,189525,189525,https://www.the-sun.com/tech/6022063/unprecede...,2022-08-17,en,'Unprecedented' artificial intelligence breakt...,'Unprecedented' artificial intelligence breakt...,Unprecedented artificial intelligence breakthr...,The US Sun,'Unprecedented' artificial intelligence breakt...,"['unprecedented', 'artificial', 'intelligence'...","[unprecedented, artificial, intelligence, brea..."
12004,12004,12004,https://www.wavy.com/news/technology/pentagon-...,2020-02-25,en,Pentagon adopts new ethical principles for usi...,\n\nPentagon adopts new ethical principles for...,Pentagon adopts new ethical principles for us...,WAVY.com,Pentagon adopts new ethical principles for usi...,"['pentagon', 'adopts', 'new', 'ethical', 'prin...","[pentagon, adopts, new, ethical, principles, a..."
108298,108298,108298,https://finance.yahoo.com/news/china-live-stre...,2022-03-25,en,"China Live Streaming E-Commerce Markets, 2022-...","China Live Streaming E-Commerce Markets, 2022-...","China Live Streaming ECommerce Markets, 202220...",Increasing Adoption of Artificial Intelligence...,"China Live Streaming E-Commerce Markets, 2022-...","['china', 'live', 'streaming', 'commerce', 'ma...","[china, live, streaming, ecommerce, markets, r..."
5447,5447,5447,https://www.computerweekly.com/news/252514030/...,2022-03-02,en,"Ban predictive policing systems in EU AI Act, ...",\nBan predictive policing systems in EU AI Act...,"Ban predictive policing systems in EU AI Act,...",None,"Ban predictive policing systems in EU AI Act, ...","['ban', 'predictive', 'policing', 'systems', '...","[ban, predictive, policing, systems, eu, ai, a..."
108869,108869,108869,https://www.newsbreak.com/news/2894700142693/a...,2023-01-22,en,A Princeton student created an app that outsma...,A Princeton student created an app that outsma...,A Princeton student created an app that outsma...,NewsBreak,A Princeton student created an app that outsma...,"['princeton', 'student', 'created', 'app', 'ou...","[princeton, student, created, app, outsmarts, ..."


In [16]:
# Getting the list of article tokens
art_tokens_samp=df_news_samp['cleaned_text_tokens'].tolist()

In [17]:
type(art_tokens_samp)

list

----
## Part A: Conducting Topic Modeling on Sample news articles using LDA

In [18]:
# Creating the term dictionary of our corpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(art_tokens_samp)
print(len(dictionary))

190163


In [19]:
token2id = dictionary.token2id
first_5_ids = list(dictionary.token2id.keys())[:30]
print(first_5_ids)

['access', 'accuracy', 'accurate', 'achieve', 'ad', 'advance', 'advanced', 'advisor', 'ago', 'ai', 'aibrave', 'ambition', 'applications', 'artemis', 'artificial', 'aug', 'author', 'az', 'based', 'baum', 'better', 'bizzare', 'blind', 'breakthrough', 'bring', 'calculations', 'california', 'called', 'capable', 'cash']


In [ ]:
token_frequencies = dictionary.dfs
print(token_frequencies)  # Output: Token frequencies in the corpus

In [22]:
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in art_tokens_samp]

In [24]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b,text):
    
    lda_model = LdaMulticore(corpus=doc_term_matrix,
                       id2word=dictionary,
                       num_topics=k,
                       random_state=100,                  
                       passes=10,
                       alpha=a,
                       eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=text, dictionary=dictionary, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [45]:
%%time

import numpy as np

grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 2
max_topics = 15
step_size = 1
topics_range = range(min_topics, max_topics+1, step_size)

alpha = ['asymmetric'] # Run for number of topics only

beta = ['auto'] # Run for number of topics only


num_of_docs = len(doc_term_matrix)
corpus_sets = [doc_term_matrix]
# corpus_title = ['75% Corpus', '100% Corpus']
corpus_title = ['100% Corpus']
model_results = {
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

itr = 0
itr_total = len(beta)*len(alpha)*len(topics_range)*len(corpus_title)
print(f'LDA will execute {itr_total} iterations')
    
# iterate through hyperparameters
for i in range(len(corpus_sets)):
    # iterate through number of topics
    for k in topics_range:
        # iterate through alpha values
        for a in alpha:
            print(a)
            # iterare through beta values
            for b in beta:
                # get the coherence score for the given parameters
                itr += 1
                cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=dictionary, 
                                              k=k, a=a, b=b, text = art_tokens_samp)
                # Save the model results
                model_results['Topics'].append(k)
                model_results['Alpha'].append(a)
                model_results['Beta'].append(b)
                model_results['Coherence'].append(cv)
                pct_completed = round((itr / itr_total * 100),1)
#                 print(f'Completed Percent: {pct_completed}%, Corpus: {corpus_title[i]}, Topics: {k}, Alpha: {a}, Beta: {b}, Coherence: {cv}')
        print(f'Completed model based on {k} LDA topics. Finished {pct_completed}% of LDA runs')
                    
lda_tuning_news_arts_samp = pd.DataFrame(model_results)

LDA will execute 14 iterations
asymmetric
Completed model based on 2 LDA topics. Finished 7.1% of LDA runs
asymmetric
Completed model based on 3 LDA topics. Finished 14.3% of LDA runs
asymmetric
Completed model based on 4 LDA topics. Finished 21.4% of LDA runs
asymmetric
Completed model based on 5 LDA topics. Finished 28.6% of LDA runs
asymmetric
Completed model based on 6 LDA topics. Finished 35.7% of LDA runs
asymmetric
Completed model based on 7 LDA topics. Finished 42.9% of LDA runs
asymmetric
Completed model based on 8 LDA topics. Finished 50.0% of LDA runs
asymmetric
Completed model based on 9 LDA topics. Finished 57.1% of LDA runs
asymmetric
Completed model based on 10 LDA topics. Finished 64.3% of LDA runs
asymmetric
Completed model based on 11 LDA topics. Finished 71.4% of LDA runs
asymmetric
Completed model based on 12 LDA topics. Finished 78.6% of LDA runs
asymmetric
Completed model based on 13 LDA topics. Finished 85.7% of LDA runs
asymmetric
Completed model based on 14 LDA

In [46]:
lda_tuning_news_arts_samp.sort_values(by=['Coherence'],ascending=False)

,Topics,Alpha,Beta,Coherence
8,10,asymmetric,auto,0.436359
0,2,asymmetric,auto,0.427764
13,15,asymmetric,auto,0.426163
9,11,asymmetric,auto,0.422421
7,9,asymmetric,auto,0.410547
3,5,asymmetric,auto,0.410254
10,12,asymmetric,auto,0.409700
4,6,asymmetric,auto,0.404761
11,13,asymmetric,auto,0.402978
5,7,asymmetric,auto,0.401551


In [47]:
%%time

best_lda_model_art_samp = LdaMulticore(corpus=doc_term_matrix,
                       id2word=dictionary,
                       num_topics=10,
                       random_state=100,
                       passes=10,
                       alpha='asymmetric',
                       eta='auto')

coherence_model_lda = CoherenceModel(model=best_lda_model_art_samp, texts=art_tokens_samp, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

CPU times: user 3min 10s, sys: 2min 20s, total: 5min 31s
Wall time: 1min 51s


In [48]:
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.43635931883055806


In [49]:
%%time

lda_display = gensimvis.prepare(best_lda_model_art_samp, doc_term_matrix, dictionary, sort_topics=False, mds='mmds')
pyLDAvis.display(lda_display)

/opt/conda/lib/python3.7/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


CPU times: user 47.9 s, sys: 50.2 s, total: 1min 38s
Wall time: 13.5 s


In [50]:
## Tuning on alpha and beta is next

In [57]:
grid = {}
grid['Validation_Set'] = {}

topics_range = [10] # Obtained from previous HP Tuning

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.5))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.5))
beta.append('symmetric')

# Validation sets
num_of_docs = len(doc_term_matrix)
corpus_sets = [doc_term_matrix]

corpus_title = ['100% Corpus']
model_results_art_samp_hp2 = {
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
itr = 0
itr_total = len(beta)*len(alpha)*len(topics_range)*len(corpus_title)
print(f'LDA will execute {itr_total} iterations')

LDA will execute 12 iterations


In [58]:
# iterate through hyperparameters
for i in range(len(corpus_sets)):
    # iterate through number of topics
    for k in topics_range:
        # iterate through alpha values
        for a in alpha:
            # iterare through beta values
            for b in beta:
                print(f'alpha value is: {a}, beta value is {b}')
                # get the coherence score for the given parameters
                itr += 1
                cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=dictionary, 
                                              k=k, a=a, b=b, text = art_tokens_samp)
                # Save the model results
                model_results_art_samp_hp2['Topics'].append(k)
                model_results_art_samp_hp2['Alpha'].append(a)
                 model_results_art_samp_hp2['Coherence'].append(cv)
                pct_completed = round((itr / itr_total * 100),1)
                print(f'Completed Iteration #{itr}')
#                 print(f'Completed Percent: {pct_completed}%, Corpus: {corpus_title[i]}, Topics: {k}, Alpha: {a}, Beta: {b}, Coherence: {cv}')
        print(f'Completed model based on {k} LDA topics. Finished {pct_completed}% of LDA runs')
                    
lda_tuning_art_samp_hp2 = pd.DataFrame(model_results_art_samp_hp2)

alpha value is: 0.01, beta value is 0.01
Completed Iteration #1
alpha value is: 0.01, beta value is 0.51
Completed Iteration #2
alpha value is: 0.01, beta value is symmetric
Completed Iteration #3
alpha value is: 0.51, beta value is 0.01
Completed Iteration #4
alpha value is: 0.51, beta value is 0.51
Completed Iteration #5
alpha value is: 0.51, beta value is symmetric
Completed Iteration #6
alpha value is: symmetric, beta value is 0.01
Completed Iteration #7
alpha value is: symmetric, beta value is 0.51
Completed Iteration #8
alpha value is: symmetric, beta value is symmetric
Completed Iteration #9
alpha value is: asymmetric, beta value is 0.01
Completed Iteration #10
alpha value is: asymmetric, beta value is 0.51
Completed Iteration #11
alpha value is: asymmetric, beta value is symmetric
Completed Iteration #12
Completed model based on 10 LDA topics. Finished 100.0% of LDA runs


In [59]:
lda_tuning_art_samp_hp2.sort_values(by=['Coherence'],ascending=False)

,Topics,Alpha,Beta,Coherence
11,10,asymmetric,symmetric,0.436359
3,10,0.51,0.01,0.435461
9,10,asymmetric,0.01,0.433394
5,10,0.51,symmetric,0.430885
0,10,0.01,0.01,0.430189
6,10,symmetric,0.01,0.430189
2,10,0.01,symmetric,0.422500
8,10,symmetric,symmetric,0.421495
1,10,0.01,0.51,0.405782
7,10,symmetric,0.51,0.405361


In [60]:
%%time

best_lda_model_art_samp_hp2 = LdaMulticore(corpus=doc_term_matrix,
                       id2word=dictionary,
                       num_topics=10,
                       random_state=100,
                       passes=10,
                       alpha=0.51,
                       eta=0.01)

coherence_model_lda_art_samp_hp2 = CoherenceModel(model=best_lda_model_art_samp_hp2, texts=art_tokens_samp, dictionary=dictionary, coherence='c_v')
coherence_lda_art_samp_hp2 = coherence_model_lda_art_samp_hp2.get_coherence()

print('\nCoherence Score: ', coherence_lda_art_samp_hp2)


Coherence Score:  0.43546141084732115
CPU times: user 3min 26s, sys: 2min 24s, total: 5min 50s
Wall time: 2min 3s


In [61]:
%%time

lda_display_art_samp_hp2 = gensimvis.prepare(best_lda_model_art_samp_hp2, doc_term_matrix, dictionary, sort_topics=False, mds='mmds')
pyLDAvis.display(lda_display_art_samp_hp2)

/opt/conda/lib/python3.7/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


CPU times: user 1min 7s, sys: 1min 9s, total: 2min 17s
Wall time: 17.4 s


---
## Checking for more number of topics

In [25]:
%%time

import numpy as np

grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 15
max_topics = 25
step_size = 1
topics_range = range(min_topics, max_topics+1, step_size)

alpha = ['asymmetric'] # Run for number of topics only
beta = ['auto'] # Run for number of topics only


num_of_docs = len(doc_term_matrix)
corpus_sets = [doc_term_matrix]
# corpus_title = ['75% Corpus', '100% Corpus']
corpus_title = ['100% Corpus']
model_results_art_samp_hp3 = {
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

itr = 0
itr_total = len(beta)*len(alpha)*len(topics_range)*len(corpus_title)
print(f'LDA will execute {itr_total} iterations')
    
# iterate through hyperparameters
for i in range(len(corpus_sets)):
    # iterate through number of topics
    for k in topics_range:
        # iterate through alpha values
        for a in alpha:
            print(a)
            # iterare through beta values
            for b in beta:
                # get the coherence score for the given parameters
                itr += 1
                cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=dictionary, 
                                              k=k, a=a, b=b, text = art_tokens_samp)
                # Save the model results
                model_results_art_samp_hp3['Topics'].append(k)
                model_results_art_samp_hp3['Alpha'].append(a)
                model_results_art_samp_hp3['Beta'].append(b)
                model_results_art_samp_hp3['Coherence'].append(cv)
                pct_completed = round((itr / itr_total * 100),1)
        print(f'Completed model based on {k} LDA topics. Finished {pct_completed}% of LDA runs')
                    
lda_tuning_news_arts_samp_hp3 = pd.DataFrame(model_results_art_samp_hp3)

LDA will execute 11 iterations
asymmetric
Completed model based on 15 LDA topics. Finished 9.1% of LDA runs
asymmetric
Completed model based on 16 LDA topics. Finished 18.2% of LDA runs
asymmetric
Completed model based on 17 LDA topics. Finished 27.3% of LDA runs
asymmetric
Completed model based on 18 LDA topics. Finished 36.4% of LDA runs
asymmetric
Completed model based on 19 LDA topics. Finished 45.5% of LDA runs
asymmetric
Completed model based on 20 LDA topics. Finished 54.5% of LDA runs
asymmetric
Completed model based on 21 LDA topics. Finished 63.6% of LDA runs
asymmetric
Completed model based on 22 LDA topics. Finished 72.7% of LDA runs
asymmetric
Completed model based on 23 LDA topics. Finished 81.8% of LDA runs
asymmetric
Completed model based on 24 LDA topics. Finished 90.9% of LDA runs
asymmetric
Completed model based on 25 LDA topics. Finished 100.0% of LDA runs
CPU times: user 1h 15min 6s, sys: 1h 5min 25s, total: 2h 20min 32s
Wall time: 36min 48s


In [26]:
lda_tuning_news_arts_samp_hp3.sort_values(by=['Coherence'],ascending=False)

,Topics,Alpha,Beta,Coherence
3,18,asymmetric,auto,0.437869
0,15,asymmetric,auto,0.426163
2,17,asymmetric,auto,0.425756
1,16,asymmetric,auto,0.424985
6,21,asymmetric,auto,0.417592
10,25,asymmetric,auto,0.402721
4,19,asymmetric,auto,0.398505
8,23,asymmetric,auto,0.398323
7,22,asymmetric,auto,0.398304
9,24,asymmetric,auto,0.395480


## Tuning on alphas and betas with 18 topics

In [29]:
grid = {}
grid['Validation_Set'] = {}

topics_range = [18] # Obtained from previous HP Tuning

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.5))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.5))
beta.append('symmetric')

# Validation sets
num_of_docs = len(doc_term_matrix)
corpus_sets = [doc_term_matrix]

corpus_title = ['100% Corpus']
model_results_art_samp_hp4 = {
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
itr = 0
itr_total = len(beta)*len(alpha)*len(topics_range)*len(corpus_title)
print(f'LDA will execute {itr_total} iterations')

LDA will execute 12 iterations


In [30]:
%%time

# iterate through hyperparameters
for i in range(len(corpus_sets)):
    # iterate through number of topics
    for k in topics_range:
        # iterate through alpha values
        for a in alpha:
            # iterare through beta values
            for b in beta:
                print(f'alpha value is: {a}, beta value is {b}')
                # get the coherence score for the given parameters
                itr += 1
                cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=dictionary, 
                                              k=k, a=a, b=b, text = art_tokens_samp)
                # Save the model results
                model_results_art_samp_hp4['Topics'].append(k)
                model_results_art_samp_hp4['Alpha'].append(a)
                model_results_art_samp_hp4['Beta'].append(b)
                model_results_art_samp_hp4['Coherence'].append(cv)
                pct_completed = round((itr / itr_total * 100),1)
                print(f'Completed Iteration #{itr}')
#                 print(f'Completed Percent: {pct_completed}%, Corpus: {corpus_title[i]}, Topics: {k}, Alpha: {a}, Beta: {b}, Coherence: {cv}')
        print(f'Completed model based on {k} LDA topics. Finished {pct_completed}% of LDA runs')
                    
lda_tuning_art_samp_hp4 = pd.DataFrame(model_results_art_samp_hp4)

alpha value is: 0.01, beta value is 0.01
Completed Iteration #1
alpha value is: 0.01, beta value is 0.51
Completed Iteration #2
alpha value is: 0.01, beta value is symmetric
Completed Iteration #3
alpha value is: 0.51, beta value is 0.01
Completed Iteration #4
alpha value is: 0.51, beta value is 0.51
Completed Iteration #5
alpha value is: 0.51, beta value is symmetric
Completed Iteration #6
alpha value is: symmetric, beta value is 0.01
Completed Iteration #7
alpha value is: symmetric, beta value is 0.51
Completed Iteration #8
alpha value is: symmetric, beta value is symmetric
Completed Iteration #9
alpha value is: asymmetric, beta value is 0.01
Completed Iteration #10
alpha value is: asymmetric, beta value is 0.51
Completed Iteration #12
Completed model based on 18 LDA topics. Finished 100.0% of LDA runs


ValueError: All arrays must be of the same length

In [32]:
model_results_art_samp_hp4

{'Topics': [18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18],
 'Alpha': [0.01,
  0.01,
  0.01,
  0.51,
  0.51,
  0.51,
  'symmetric',
  'symmetric',
  'symmetric',
  'asymmetric',
  'asymmetric',
  'asymmetric'],
 'Beta': [],
 'Coherence': [0.42596749054407845,
  0.41810588292610573,
  0.4384226837712182,
  0.42907838069375054,
  0.39864761353850997,
  0.42727887449560875,
  0.42596749054407845,
  0.4192373173146608,
  0.43621743688378295,
  0.4235834488141836,
  0.4349787837422347,
  0.437869090293488]}

In [38]:
Beta=[0.01,0.51,'symmetric',0.01,0.51,'symmetric',0.01,0.51,'symmetric',0.01,0.51,'symmetric']

In [41]:
model_results_art_samp_hp4['Beta']=Beta
model_results_art_samp_hp4
lda_tuning_art_samp_hp4 = pd.DataFrame(model_results_art_samp_hp4)
lda_tuning_art_samp_hp4.sort_values(by=['Coherence'],ascending=False)

,Topics,Alpha,Beta,Coherence
2,18,0.01,symmetric,0.438423
11,18,asymmetric,symmetric,0.437869
8,18,symmetric,symmetric,0.436217
10,18,asymmetric,0.51,0.434979
3,18,0.51,0.01,0.429078
5,18,0.51,symmetric,0.427279
0,18,0.01,0.01,0.425967
6,18,symmetric,0.01,0.425967
9,18,asymmetric,0.01,0.423583
7,18,symmetric,0.51,0.419237


In [42]:
# 18 topics, alpha = 0.01, beta=symmetric